TODO: script description

Script that 
* takes as input the files config-layers-X (X: polygon, point, linestring), and municipality codes
* for all municipality codes jointly, fetches the *raw* data from GeoFA and makes a *processed* network (`network/raw/` and `network/processed/`)
* for each X,
    * determine evaluation layers
    * fetch and merge gdfs for given muni (if they exist)
    * save to `input-for-bike-node-planner/X`

In [ ]:
# import libraries
import os
import shutil
import yaml
import geopandas as gpd
import pandas as pd
from shapely.geometry import LineString

exec(open("../src/raw-to-processed.py").read())

# define helper function to clean data folders
def remove_output_data(output_folders, remove_previous_output: bool = False, verbose: bool = False):

    if remove_previous_output:
        for f in output_folders:
            if os.path.exists(f):
                shutil.rmtree(f)

                os.makedirs(f)
    if verbose:
        print("Data folder cleaned!")

print("Libraries and functions imported!")

***

**Import configurations**

In [ ]:
# read config files
config = yaml.load(
    open("../config.yml"), 
    Loader=yaml.FullLoader)
proj_crs = config["proj_crs"]
wfs_version = config["geofa_wfs_version"]
node_layer_name = config["geofa_nodes_layer_name"]
stretches_layer_name = config["geofa_stretches_layer_name"]

municipalities = yaml.load(
    open("../config-municipalities.yml"), 
    Loader=yaml.FullLoader)
codes = municipalities["kommunekode"]

geomtypes = ["point", "linestring", "polygon"]
config_layers = {}
for geomtype in geomtypes:
    config_layers[geomtype] = yaml.load(
        open(f"../config-layers-{geomtype}.yml"), 
        Loader=yaml.FullLoader)
    
print("Configurations imported!")

***

**Create subfolders**

In [ ]:
# make folders
main_folder = "../input-for-bike-node-planner/"
os.makedirs(main_folder, exist_ok=True)

sub_folders = [
    "dem",
    "elevation",
    "linestring",
    "network",
    "point",
    "polygon",
    "studyarea"
]

for sub_folder in sub_folders:
    os.makedirs(main_folder + sub_folder, exist_ok=True)

print("Subfolders created!")

***

**Clean data folder**

Removing old data (if any)

In [ ]:
# remove previous output
remove_output_data(
    [
        "../input-for-bike-node-planner/dem",
        "../input-for-bike-node-planner/elevation",
        "../input-for-bike-node-planner/linestring/",
        "../input-for-bike-node-planner/network/",
        "../input-for-bike-node-planner/point/",
        "../input-for-bike-node-planner/polygon/",
        "../input-for-bike-node-planner/studyarea/"        
    ],
    remove_previous_output=True,
    verbose=True
)

***

**Create study area polygon**

(based on municipality boundaries)

In [ ]:
### read in municipality boundaries & create study area polygon
gdf = gpd.read_file("../data/municipality-boundaries/municipality-boundaries.gpkg")
gdf = gdf.to_crs(proj_crs) # make sure we have the right projected CRS
gdf = gdf[gdf["kommunekode"].isin(codes)] # filter to municipality codes indicated in config file
gdf_studyarea = gpd.GeoDataFrame(
    {
        "geometry": [gdf.unary_union]
    },
    crs = proj_crs
)
gdf_studyarea.to_file(
    filename = "../input-for-bike-node-planner/studyarea/studyarea.gpkg", 
    index = False)
print("Study area polygon created!")
#del gdf

***

**Fetch and save raw network data from GeoFA**

In [ ]:
# Fetch input data from GeoFA (raw data)

url = f"https://geofa.geodanmark.dk/ows/fkg/fkg/?request=GetFeature&typename={node_layer_name}&service=WFS&version={wfs_version}"

knudepunkter = gpd.read_file(url)

url = f"https://geofa.geodanmark.dk/ows/fkg/fkg/?request=GetFeature&typename={stretches_layer_name}&service=WFS&version={wfs_version}"

straekninger = gpd.read_file(url)

assert len(knudepunkter) > 0, "No nodes found"
assert len(straekninger) > 0, "No stretches found"

# limit to extent of study area
assert straekninger.crs == gdf_studyarea.crs
assert knudepunkter.crs == gdf_studyarea.crs

edges_studyarea = straekninger.sjoin(gdf_studyarea, predicate="intersects").copy()
edges_studyarea.drop(columns=["index_right"], inplace=True)
nodes_studyarea = knudepunkter.clip(edges_studyarea.buffer(500).unary_union)

# remove empty geometries
edges_studyarea = edges_studyarea[edges_studyarea.geometry.notna()].reset_index(
    drop=True
)
nodes_studyarea = nodes_studyarea[nodes_studyarea.geometry.notna()].reset_index(
    drop=True
)

# assert there is one (and only one) LineString per edge geometry row
nodes_studyarea = nodes_studyarea.explode(index_parts=False).reset_index(drop=True)
assert all(nodes_studyarea.geometry.type == "Point")
assert all(nodes_studyarea.geometry.is_valid)

# assert there is one (and only one) Point per node geometry row
edges_studyarea = edges_studyarea.explode(index_parts=False).reset_index(drop=True)
assert all(edges_studyarea.geometry.type == "LineString")
assert all(edges_studyarea.geometry.is_valid)

# save
os.makedirs(
    "../input-for-bike-node-planner/network/raw/", 
    exist_ok = True
)

edges_studyarea.to_file(
    "../input-for-bike-node-planner/network/raw/edges.gpkg", index=False
)

nodes_studyarea.to_file(
    "../input-for-bike-node-planner/network/raw/nodes.gpkg", index=False
)

print("Raw data on nodes and edges for study area fetched and saved!")

***

**Process (clean) and save network data**

In [ ]:
edges_studyarea["edge_id"] = edges_studyarea.id_cykelknudepunktsstraekning
assert len(edges_studyarea) == len(edges_studyarea["edge_id"].unique())

nodes_studyarea["node_id"] = nodes_studyarea.id_cykelknudepkt
assert len(nodes_studyarea) == len(nodes_studyarea["node_id"].unique())

processed_edges = assign_edges_start_end_nodes(edges_studyarea, nodes_studyarea)

processed_edges = order_edge_nodes(processed_edges)

processed_edges = find_parallel_edges(processed_edges)

assert len(processed_edges) == len(edges_studyarea)

processed_nodes = nodes_studyarea.loc[
    nodes_studyarea["node_id"].isin(processed_edges["u"])
    | nodes_studyarea["node_id"].isin(processed_edges["v"])
]

# save to files
os.makedirs(
    "../input-for-bike-node-planner/network/processed/",
    exist_ok=True
)

processed_nodes.to_file(
    "../input-for-bike-node-planner/network/processed/nodes.gpkg", index=False
)

processed_edges.to_file(
    "../input-for-bike-node-planner/network/processed/edges.gpkg", index=False
)

print("Data on nodes and edges for study area processed and saved!")

***

**Generate evaluation layers (point, linestring, polygon)**

In [ ]:
# create a dictionary of evaluation layers (based on config file inputs)

layer_dict = {}

for geomtype in geomtypes:

    layer_dict[geomtype] = {}

    # determine evaluation layers
    layers = []
    for v in config_layers[geomtype].values():
        layers += list(set(v.values()))
    layers = list(set(layers))
    
    # determine data sets that go into each layer

    # key is name of merged output layer, value is a dict 
    for layer in layers:
        layer_dict[geomtype][layer] = {}
    
    # adding data source as key to dictindict IF relevant to layer
    for datasource, vdict in config_layers[geomtype].items():
        for layer in (set(vdict.values())):
            layer_dict[geomtype][layer][datasource] = []
    
    for datasource, vdict in config_layers[geomtype].items():
        for k, v in vdict.items():
            layer_dict[geomtype][v][datasource] += [k]

for geomtype in geomtypes:
    if "ignore" in layer_dict[geomtype]:
        del layer_dict[geomtype]["ignore"]

# for each layer type (point/linestring/polygon),

print("Generation of evaluation layers started")

for geomtype in geomtypes:
    
    # go through all evaluation layers for that geomtype... 
    for layername, datadict in layer_dict[geomtype].items():
    
        final_gdf = gpd.GeoDataFrame()
        
        # go through each data source for that evaluation layer...
        for k, v in datadict.items():

            gdf = gpd.GeoDataFrame()

            # and fetch it for each municipality
            for code in codes:
                # for each code, check if file exists, if yes: read it in, if not empty: concatenate
                fp = f"../data/{geomtype}/{code}/{k}.gpkg"
                if os.path.exists(fp):
                    gdf_muni = gpd.read_file(fp)
                    if not gdf_muni.empty:
                        gdf = pd.concat(
                            [
                                gdf,
                                gdf_muni
                            ]
                        )

            # if at least one of the municipalities has data from this data source,
            # add it to final gdf
            if not gdf.empty:
                final_gdf = pd.concat(
                    [
                        final_gdf,
                        gdf[gdf["type"].isin(v)]
                    ]
                )

        # save evaluation layer to file, if not empty
        if not final_gdf.empty:
            final_gdf = final_gdf.reset_index(drop=True)
            final_gdf.to_file(
                f"../input-for-bike-node-planner/{geomtype}/{layername}.gpkg", 
                index = False
            )
            print("\t \t", f"{layername} layer saved")
        else:
            print("\t \t", f"No data found for {layername} layer")
    
print("Generation of evaluation layers ended successfully!")

***

**Download and merge elevation data**

In [ ]:
remove_output_data(
    [
        "../data/dem", 
        "../input-for-bike-node-planner/dem"
    ], 
    remove_previous_output=True
)

print("Downloading and merging elevation data. This can take up to several minutes per municipality.")

exec(open("../src/dem-download.py").read())

print("\t Elevation data downloaded!")
print("\t Merging data...")

exec(open("../src/dem-merge.py").read())

print("Elevation data ready!")

***

**All done!**

All done! Now you can copy-paste all subfolders of `/input-for-bike-node-planner/` into the `/data/input/` folder of `bike-node-planner`.

In [ ]:
print("All done! Now you can copy-paste \nall subfolders of '/input-for-bike-node-planner/' \ninto the '/data/input/' folder of bike-node-planner")